In [1]:
import helper.binance_api as ba
import helper.set_scrape as ss

In [2]:
print(ba.list_of_symbols)
print(ss.list_of_symbols)

['1INCHBTC', '1INCHBUSD', '1INCHDOWNUSDT', '1INCHUPUSDT', '1INCHUSDT', 'AAVEBKRW', 'AAVEBNB', 'AAVEBRL', 'AAVEBTC', 'AAVEBUSD', 'AAVEDOWNUSDT', 'AAVEETH', 'AAVEUPUSDT', 'AAVEUSDT', 'ACABTC', 'ACABUSD', 'ACAUSDT', 'ACHBTC', 'ACHBUSD', 'ACHUSDT', 'ACMBTC', 'ACMBUSD', 'ACMUSDT', 'ADAAUD', 'ADABIDR', 'ADABKRW', 'ADABNB', 'ADABRL', 'ADABTC', 'ADABUSD', 'ADADOWNUSDT', 'ADAETH', 'ADAEUR', 'ADAGBP', 'ADAPAX', 'ADARUB', 'ADATRY', 'ADATUSD', 'ADAUPUSDT', 'ADAUSDC', 'ADAUSDT', 'ADXBNB', 'ADXBTC', 'ADXBUSD', 'ADXETH', 'ADXUSDT', 'AEBNB', 'AEBTC', 'AEETH', 'AERGOBTC', 'AERGOBUSD', 'AGIBNB', 'AGIBTC', 'AGIETH', 'AGIXBTC', 'AGIXBUSD', 'AGLDBNB', 'AGLDBTC', 'AGLDBUSD', 'AGLDUSDT', 'AIONBNB', 'AIONBTC', 'AIONBUSD', 'AIONETH', 'AIONUSDT', 'AKROBTC', 'AKROBUSD', 'AKROUSDT', 'ALCXBTC', 'ALCXBUSD', 'ALCXUSDT', 'ALGOBIDR', 'ALGOBNB', 'ALGOBTC', 'ALGOBUSD', 'ALGOETH', 'ALGOPAX', 'ALGORUB', 'ALGOTRY', 'ALGOTUSD', 'ALGOUSDC', 'ALGOUSDT', 'ALICEBIDR', 'ALICEBNB', 'ALICEBTC', 'ALICEBUSD', 'ALICETRY', 'ALICEUSDT'

In [3]:
df_ptt = ss.get_data_1d('PTT').add_prefix('ptt_')
df_ptt

,ptt_open,ptt_high,ptt_low,ptt_close,ptt_volume
date,,,,,
2001-12-06,1.560447,1.570713,1.457786,1.468052,1736808000
2001-12-07,1.488584,1.488584,1.457786,1.457786,412533000
2001-12-10,1.457786,1.457786,1.457786,1.457786,0
2001-12-11,1.457786,1.457786,1.396189,1.396189,409158000
2001-12-12,1.406455,1.447520,1.396189,1.437254,307291000
...,...,...,...,...,...
2023-01-27,33.250000,33.500000,33.000000,33.250000,27506500
2023-01-30,33.250000,33.500000,33.250000,33.250000,18635300
2023-01-31,33.250000,33.500000,33.000000,33.250000,35789700


In [4]:
df_top = ss.get_data_1d('TOP').add_prefix('yf_top_')
df_top

,yf_top_open,yf_top_high,yf_top_low,yf_top_close,yf_top_volume
date,,,,,
2004-10-26,18.342816,19.434651,17.578532,19.216284,3083006
2004-10-27,18.670375,18.888742,18.124457,18.233641,136964100
2004-10-28,18.452006,18.561189,17.250988,17.469355,126021400
2004-10-29,17.250989,17.906090,17.032622,17.687723,41203700
2004-11-01,17.796907,18.015274,17.578540,17.796907,19119800
...,...,...,...,...,...
2023-01-27,58.250000,60.250000,58.250000,60.250000,16385500
2023-01-30,59.500000,60.000000,59.000000,59.750000,9696200
2023-01-31,59.500000,59.750000,57.750000,57.750000,13121500


In [5]:
from math import log, exp
import numpy as np
import pandas as pd
def backtest(df, func, **kwargs):
    df = df.copy()
    df['return_bah'] = (df['close'] / df['close'].shift(1)).map(log)
    df['position'] = func(df, **kwargs)
    df['return_strategy'] = df['position'].shift(1) * df['return_bah']
    df.dropna(inplace=True)
    df['creturn_bah'] = df['return_bah'].cumsum().apply(np.exp)
    df['creturn_strategy'] = df['return_strategy'].cumsum().apply(np.exp)
    # buy-and-hold returns
    perf_bah = df['creturn_bah'].iloc[-1]
    # gross performance of the strategy
    perf_gross = df['creturn_strategy'].iloc[-1]
    # out-/underperformance of strategy
    perf_net = perf_gross - df['creturn_bah'].iloc[-1]
    
    dictionary = {'df_shape': df.shape, 'buy-and-hold returns': perf_bah, 'gross performance': perf_gross, 'net performance': perf_net}

    return pd.Series(dictionary)
    

In [6]:
def strategy_sma_1asset(df, sma1 = None, sma2 = None):
    if sma1 is not None:
        df['sma1'] = df['close'].rolling(sma1).mean()
    if sma2 is not None:
        df['sma2'] = df['close'].rolling(sma2).mean()
    return np.where(df['sma1'] > df['sma2'], 1, -1)

In [8]:
# backtest(df_ptt, strategy_sma_1asset, **dict(sma1=70, sma2=160))

In [ ]:
def backtest_masset(list_of_df, list_of_prefixes, func = None, **kwargs):
    for df in list_of_df:
        df['return_bah'] = (df['close'] / df['close'].shift(1)).map(log)
        
    print(pd.concat(list_of_df, axis=1))

In [ ]:
list_of_df = [df_yf_ptt, df_yf_top]
list_of_prefixes = ['ptt_', 'top_']
backtest_masset(list_of_df, list_of_prefixes)